In [5]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_pickle("../../data/train.pkl")

In [3]:
#pull out y values
y_train = train['isFraud'].copy()
X_train = train.drop(columns=['isFraud'])
del train
gc.collect()

7

In [ ]:
#Quick LightGBM model
num_leaves = 25
feature_fraction = .2
bagging_fraction = .2
max_depth = 5
lambda_l1 = 1
lambda_l2 = 1
min_split_gain = .05
min_child_weight = 10

params = {'application':'binary','num_iterations':4000, 'learning_rate':0.05, 'early_stopping_round':100, 'metric':'auc'}
params["num_leaves"] = round(num_leaves)
params['feature_fraction'] = max(min(feature_fraction, 1), 0)
params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
params['max_depth'] = round(max_depth)
params['lambda_l1'] = max(lambda_l1, 0)
params['lambda_l2'] = max(lambda_l2, 0)
params['min_split_gain'] = min_split_gain
params['min_child_weight'] = min_child_weight

categorical_list = ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9','id_12', 'id_15', 'id_16', 'id_27', 
                     'id_28', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType']

train_data = lgb.Dataset(data=X_train, label=y_train, categorical_feature = categorical_list, free_raw_data=False)

lgb.cv(params, train_data, nfold=5, stratified=True, verbose_eval =200, metrics=['auc'])

In [6]:
estimator = lgb.LGBMClassifier()

param_grid = {
    'learning_rate': [0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 20}
